# Financial Data #

## Data Collection ##

In [159]:
# lib imports
import pandas as pd
import requests
import yfinance as YF
from fredapi import Fred


In [160]:
# define key variables
fred_key = "7c1f3f043ef85e198d4c6c17e0d8109e"
eia_key = "x7uY3CJRn9HYOS6gtcMsMAllomMfCLGvWYYyn8Cz"
tickers = ['AAPL', 'PLTR', 'WMT', 'META', 'AMZN', 'LMT']

# func to obtain stock data info from yahoo finance
def get_tickers(tickers=[], period='1d'):
    # get tickers from yahoo finance
    table = pd.DataFrame()
    for ticker in tickers:
        data = YF.Ticker(ticker)
        df = data.history(period=period)
        df['ticker'] = ticker
        table = pd.concat([table, df])
    return table

In [161]:
# pull stocks data
stocks = get_tickers(tickers, period='max')
stocks.reset_index(inplace=True)

In [162]:
# pull fred data
fred_api = Fred(fred_key)
cpi = fred_api.get_series('CPIAUCNS')
unrate = fred_api.get_series("Unrate")
csi = fred_api.get_series("UMCSENT")

In [163]:
# join fred data
fred = pd.DataFrame(pd.concat([cpi, unrate, csi], axis=1)).reset_index()
fred.rename(columns={'index': 'Date', 0: 'cpi', 1: 'unrate', 2:'csi'}, inplace=True)

In [164]:
# Convert stock dates to naive datetime (removing timezone)
stocks['Date'] = stocks['Date'].dt.tz_localize(None)

# Convert fred dates to ensure consistency (though likely already naive)
fred['Date'] = fred['Date'].dt.tz_localize(None)

# merge data on date
fi_df = pd.merge(stocks, fred, on='Date', how='left')

# backfill monthly indicators to prevent NaN values
fi_df[['cpi', 'unrate', 'csi']] = fi_df[['cpi', 'unrate', 'csi']].ffill()
fi_df[['cpi', 'unrate', 'csi']] = fi_df[['cpi', 'unrate', 'csi']].bfill()

# ensure consistency
fi_df.columns = fi_df.columns.str.lower()
fi_df.head()

,date,open,high,low,close,volume,dividends,stock splits,ticker,cpi,unrate,csi
0,1980-12-12,0.098726,0.099155,0.098726,0.098726,469033600,0.0,0.0,AAPL,89.1,7.2,72.4
1,1980-12-15,0.094005,0.094005,0.093575,0.093575,175884800,0.0,0.0,AAPL,89.1,7.2,72.4
2,1980-12-16,0.087136,0.087136,0.086707,0.086707,105728000,0.0,0.0,AAPL,89.1,7.2,72.4
3,1980-12-17,0.088853,0.089282,0.088853,0.088853,86441600,0.0,0.0,AAPL,89.1,7.2,72.4
4,1980-12-18,0.091429,0.091858,0.091429,0.091429,73449600,0.0,0.0,AAPL,89.1,7.2,72.4


## Push into Database ##